In [1]:
from typing import Annotated, Literal
import os
from autogen import ConversableAgent, UserProxyAgent


/home/gilbe/ai_assistant/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Start using Tools

In [2]:
Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return a / b
    else:
        raise ValueError("Invalid operator")


# Registering Tools

In [3]:
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations."
    "Return 'TERMINATE' when the task is done.",
    llm_config={
        "config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
    },
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)
assistant.register_for_llm(
    name="calculator",
    description="A simple calculator",
)(calculator)

user_proxy.register_for_execution(name="calculator")(calculator)


<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

# Using the Tool

In [4]:
chat_result = user_proxy.initiate_chat(
    assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?"
)


User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_uvumeLpoaF9Le2n1dEmRzK3r): calculator *****
Arguments: 
{"a":13312,"b":200,"operator":"/"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_uvumeLpoaF9Le2n1dEmRzK3r) *****
66.56
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_EsdBz5QwwuTCq6J7iKnqIxtf): calculator *****
Arguments: 
{"a":44232,"b":66.56,"operator":"+"}
***************************

# Local Model: Ollam + liteLLM ---> llama3

In [6]:
local_llm_config = {
    "config_list": [
        {
            "model": "NotRequired",
            "api_key": "NotRequired",
            "base_url": "http://0.0.0.0:4000",
        }
    ],
    "cached_seed": None,
}

local_assistant = ConversableAgent("agent", llm_config=local_llm_config)
user_proxy = UserProxyAgent("user", code_execution_config=False)
assistant.initiate_chat(user_proxy, message="How can I help you today?")


Assistant (to user):

How can I help you today?

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
user (to Assistant):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to user):

It appears you haven't provided any details. How can I assist you today?

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
user (to Assistant):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to user):

If you need assistance, feel free to ask! What can I help you with today?

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
user (to Assistant):



--------------------------